In [3]:
import math


118264581564861424

In [2]:
def ncr(n, r):
    # if python version == 3.8+, use comb()
    if r == 0:
        return 1
    return n * ncr(n-1, r-1) // r

In [4]:
ncr(60,30)

118264581564861424

In [7]:
ncr(59,30) + ncr(59,30)

118264581564861424

In [8]:
ncr(3,2)

3

In [10]:
math.log(2*10**5)/math.log(2)

17.609640474436812